In [1]:
from torchsummary import summary
import torch

In [2]:
from conti_model import MultiSeqBase
model = MultiSeqBase(hidden_size = 16, embedding_dim = 4, dropout_p = 0.2)
optim = torch.optim.Adam(model.parameters(), weight_decay=1e-3)

checkpoint = torch.load('test_model_and_optimizer_conti.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optim.load_state_dict(checkpoint['optimizer_state_dict'])

AttributeError: 'MultiSeqBase' object has no attribute 'save'

In [1]:
from utils import *
   
r_seq = db2Rseq()
h_seq = db2Hseq(h_step = 4, pred_step = 6)
t = dt2T(pred_step = 6)
s_attrs, _ = db2S()

# output = model(r_seq.float(), h_seq.float(), t.int(), s_attrs.float())

RuntimeError: Error(s) in loading state_dict for MultiSeqBase:
	size mismatch for lstm_r1.weight_ih_l0: copying a param with shape torch.Size([64, 1]) from checkpoint, the shape in current model is torch.Size([128, 1]).
	size mismatch for lstm_r1.weight_hh_l0: copying a param with shape torch.Size([64, 16]) from checkpoint, the shape in current model is torch.Size([128, 32]).
	size mismatch for lstm_r1.bias_ih_l0: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for lstm_r1.bias_hh_l0: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([128]).
	size mismatch for fc_r1.weight: copying a param with shape torch.Size([16, 16]) from checkpoint, the shape in current model is torch.Size([16, 32]).
	size mismatch for lstm_h1.weight_ih_l0: copying a param with shape torch.Size([64, 1]) from checkpoint, the shape in current model is torch.Size([128, 1]).
	size mismatch f

In [5]:
# test for lambda
import time
import datetime

now = time.time()
connection = db_connect()
select_query = "SELECT * FROM occupancy"

with connection.cursor() as cursor:
        cursor.execute(select_query)
        result = cursor.fetchall()
        connection.close()

df = pd.DataFrame(result, columns=[col[0] for col in cursor.description])

print('running time:', time.time() - now)
print('time:', now, 'summation:', df.sum().sum())

running time: 0.08759307861328125
time: 1692321029.1656742 summation: 369411
